In [160]:
from src.data import DATA_DIR
import pandas as pd

In [161]:
df = pd.read_excel(DATA_DIR / 'SP1.xlsx')

df['Score HomeTeam'] = 0
df['Score AwayTeam'] = 0

df.loc[df.FTHG > df.FTAG, 'Score HomeTeam'] = 3
df.loc[df.FTHG < df.FTAG, 'Score AwayTeam'] = 3
df.loc[df.FTHG == df.FTAG, ['Score HomeTeam', 'Score AwayTeam']] = 1

df.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Score HomeTeam,Score AwayTeam
0,SP1,1993-09-05,Ath Bilbao,Albacete,4,1,H,3,0
1,SP1,1993-09-05,Ath Madrid,Logrones,1,0,H,3,0
2,SP1,1993-09-05,Barcelona,Sociedad,3,0,H,3,0
3,SP1,1993-09-05,La Coruna,Celta,0,0,D,1,1
4,SP1,1993-09-05,Osasuna,Real Madrid,1,4,A,0,3


In [162]:
list_ = []

for i in range(len(df)):
    if df.iloc[i, 7] > 0:
        list_.append((df.iloc[i, 1], df.iloc[i, 2], df.iloc[i, 7]))
    
    if df.iloc[i, 8] > 0:
        list_.append((df.iloc[i, 1], df.iloc[i, 3], df.iloc[i, 8]))

In [163]:
final_df = pd.DataFrame(list_, columns=['Date', 'Team', 'Score']).pivot(index='Date', columns='Team', values='Score')

In [164]:
final_df = final_df.fillna(0)

for i in range(1, len(final_df)):
    final_df.iloc[i, :] += final_df.iloc[i - 1, :]

In [165]:
dates = set([(date.year, date.month) for date in final_df.index])
dates = sorted(set(dates))

df_ = []
for i in range(len(dates)-1):
    df = final_df.loc[(final_df.index < datetime.datetime(dates[i+1][0], dates[i+1][1], 1, 0, 0)) &
                      (final_df.index >= datetime.datetime(dates[i][0], dates[i][1], 1, 0, 0))]
    
    # df['Date'] = datetime.datetime(dates[i+1][0], dates[i+1][1], 1, 0, 0)
    
    if not len(df):
        continue
    
    if not len(df_):
        df_ = df.loc[df.index == max(df.index)]
        
    else:
        df_ = pd.concat([df_, df.loc[df.index == max(df.index)]])

In [166]:
from dateutil.relativedelta import relativedelta

df_['Date'] = None

for i in range(len(df_)):
    df_.iloc[i, 49] = datetime.datetime(df_.index[i].year, df_.index[i].month, 1, 0, 0) + relativedelta(months=1)

In [167]:
df_ = df_.set_index('Date')

In [168]:
df_.to_csv(DATA_DIR / 'Final_dataset.csv')